In [1]:
# Fix for issue loading Utils.preprocess_util
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import torch
import torch.nn as nn
import torch.optim  as optim
from torch.autograd import Variable
from Utils.preprocess_util import *
from Utils.visualize import *

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2


In [2]:
X_train,X_valid,X_test,Y_train,Y_valid,Y_test = load_preprocess_eeg_data()


(2115, 22, 1000)
Cropping trials
(177125, 22, 500)
After cropping:
Training data: (177125, 22, 500)
Training target: (177125,)
Validation data: (87250, 22, 500)
Validation target: (87250,)
Test data: (55375, 22, 500)
Test target: (55375,)
Person train/validation: (2115, 1)
Person test: (443, 1)

After cropping:
Training data: (177125, 22, 500)
Training target: (177125,)
Validation data: (87250, 22, 500)
Validation target: (87250,)
Test data: (55375, 22, 500)
Test target: (55375,)
Person train/validation: (2115, 1)
Person test: (443, 1)



In [3]:
class Flatten(nn.Module):
    def forward(self, x):
        a= x.view(x.size(0), -1)
        return a
    
class threed_to_twod(nn.Module):
    def forward(self, x):
        # print(x.shape)
        a = x.reshape(x.shape[0],x.shape[3],x.shape[1])
        # print (a)
        return a

In [4]:
model = nn.Sequential()
model.add_module('conv_across_time',nn.Conv2d(1,40,kernel_size=(1,51),stride = 1))
model.add_module('conv_across_electrodes',nn.Conv2d(40,40,kernel_size=(22,1),stride = 1))
model.add_module('BatchNorm2d',nn.BatchNorm2d(40,momentum=0.1))
model.add_module('Nonlinearity', nn.ReLU())
model.add_module('correct_dimensions',threed_to_twod())
model.add_module('AvgPool2d',nn.AvgPool2d(kernel_size=(135,1),stride = (5,1)))
model.add_module('drop', nn.Dropout(p=0.5))
model.add_module('Flatten',Flatten())    
model.add_module('Fc_layer',nn.Linear(2560,10))
torch.nn.init.xavier_uniform_(model.conv_across_time.weight, gain=1)
torch.nn.init.xavier_uniform_(model.conv_across_electrodes.weight, gain=1)


Parameter containing:
tensor([[[[ 0.0141],
          [-0.0128],
          [ 0.0456],
          ...,
          [-0.0340],
          [ 0.0013],
          [ 0.0537]],

         [[-0.0150],
          [ 0.0414],
          [-0.0109],
          ...,
          [-0.0090],
          [ 0.0045],
          [-0.0065]],

         [[ 0.0275],
          [ 0.0417],
          [ 0.0163],
          ...,
          [-0.0120],
          [ 0.0476],
          [ 0.0289]],

         ...,

         [[ 0.0410],
          [ 0.0370],
          [-0.0467],
          ...,
          [ 0.0365],
          [ 0.0146],
          [ 0.0126]],

         [[-0.0332],
          [ 0.0429],
          [ 0.0522],
          ...,
          [ 0.0035],
          [ 0.0080],
          [-0.0157]],

         [[ 0.0154],
          [-0.0466],
          [ 0.0372],
          ...,
          [ 0.0340],
          [-0.0122],
          [-0.0518]]],


        [[[-0.0456],
          [-0.0409],
          [-0.0253],
          ...,
          [ 0.0014],
    

In [6]:
dtype = torch.FloatTensor
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)


In [ ]:
# N,C,H,W = 18,1,25,1000
# x = Variable(torch.tensor(X_train.reshape((18,1, 25, 1000))))
x = Variable(torch.tensor(X_train))
y = Variable(torch.tensor(Y_train),requires_grad=False)
dtype = torch.FloatTensor
x.type(dtype)
y.type(dtype)

In [13]:
'''
for t in range(3):
    y_pred = model( x.float())
    loss = loss_fn(y_pred,y.type(torch.LongTensor))
    print(loss.data)
    model.zero_grad()
    loss.backward()
loss = loss_fn(y_pred,y.type(torch.LongTensor))
'''

'\nfor t in range(3):\n    y_pred = model( x.float())\n    loss = loss_fn(y_pred,y.type(torch.LongTensor))\n    print(loss.data)\n    model.zero_grad()\n    loss.backward()\nloss = loss_fn(y_pred,y.type(torch.LongTensor))\n'

In [8]:
num_train = X_train.shape[0]
num_valid = X_valid.shape[0]
batch_size = 100
num_epochs = 50
iterations_per_epoch = max(num_train // batch_size, 1)
num_iterations = num_epochs * iterations_per_epoch
epoch = 1
train_loss = []
iterations = []

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)   
for t in range(num_iterations):
    batch_mask = np.random.choice(num_train, batch_size)
    X_batch = X_train[batch_mask]
    y_batch = Y_train[batch_mask]
    X_batch_tensor = threeD_to_fourDTensor(X_batch)
    y_batch_tensor = Variable(torch.tensor(y_batch))
    
    y_pred = model( X_batch_tensor.float())
    
    loss = loss_fn(y_pred,y_batch_tensor.type(torch.LongTensor))
    
    train_loss.append(loss.data)
    iterations.append(t)
        
    model.zero_grad()
    loss.backward()
    optimizer.step()
    if(t%10 == 0):
        print('(Iteration %d / %d) loss: %f' % (
                       t + 1, num_iterations, loss.detach().numpy()))
    
    epoch_end = (t + 1) % iterations_per_epoch == 0
    
    if epoch_end:
                epoch += 1
    
    first_it = (t == 0)
    last_it = (t == num_iterations - 1)

    if first_it or last_it or epoch_end:
        X_train_tensor =threeD_to_fourDTensor(X_train[0:100,:,:])
        y_pred_train = model( X_train_tensor.float())
        train_acc = get_accuracy(y_pred_train, Y_train[0:100],
            batch_size=50)
        
        X_valid_tensor = threeD_to_fourDTensor(X_valid[0:50,:,:])
        y_pred_valid = model( X_valid_tensor.float())
        val_acc = get_accuracy(y_pred_valid, Y_valid[0:50],
            batch_size=50)
        print('(Epoch %d / %d) train acc: %f; val_acc: %f' % (
                           epoch, num_epochs, train_acc, val_acc))


(Iteration 1 / 88550) loss: 2.335279
(Epoch 1 / 50) train acc: 14.000000; val_acc: 18.000000
(Iteration 11 / 88550) loss: 1.873507
(Iteration 21 / 88550) loss: 1.633958
(Iteration 31 / 88550) loss: 1.524756
(Iteration 41 / 88550) loss: 1.470626
(Iteration 51 / 88550) loss: 1.440654
(Iteration 61 / 88550) loss: 1.425282
(Iteration 71 / 88550) loss: 1.436383
(Iteration 81 / 88550) loss: 1.411892
(Iteration 91 / 88550) loss: 1.410616
(Iteration 101 / 88550) loss: 1.400183
(Iteration 111 / 88550) loss: 1.383582
(Iteration 121 / 88550) loss: 1.410469
(Iteration 131 / 88550) loss: 1.408475
(Iteration 141 / 88550) loss: 1.394894
(Iteration 151 / 88550) loss: 1.403960
(Iteration 161 / 88550) loss: 1.409156
(Iteration 171 / 88550) loss: 1.388676
(Iteration 181 / 88550) loss: 1.396498
(Iteration 191 / 88550) loss: 1.411703
(Iteration 201 / 88550) loss: 1.407684
(Iteration 211 / 88550) loss: 1.386379
(Iteration 221 / 88550) loss: 1.413196
(Iteration 231 / 88550) loss: 1.362272
(Iteration 241 / 88

(Iteration 2051 / 88550) loss: 1.119657
(Iteration 2061 / 88550) loss: 1.203485
(Iteration 2071 / 88550) loss: 1.146266
(Iteration 2081 / 88550) loss: 1.138059
(Iteration 2091 / 88550) loss: 1.150768
(Iteration 2101 / 88550) loss: 1.112139
(Iteration 2111 / 88550) loss: 1.246979
(Iteration 2121 / 88550) loss: 1.115684
(Iteration 2131 / 88550) loss: 1.040018
(Iteration 2141 / 88550) loss: 1.195043
(Iteration 2151 / 88550) loss: 1.178815
(Iteration 2161 / 88550) loss: 1.242809
(Iteration 2171 / 88550) loss: 1.100996
(Iteration 2181 / 88550) loss: 1.144193
(Iteration 2191 / 88550) loss: 1.088432
(Iteration 2201 / 88550) loss: 1.139265
(Iteration 2211 / 88550) loss: 1.112007
(Iteration 2221 / 88550) loss: 1.109902
(Iteration 2231 / 88550) loss: 1.142923
(Iteration 2241 / 88550) loss: 1.121666
(Iteration 2251 / 88550) loss: 1.130620
(Iteration 2261 / 88550) loss: 1.074307
(Iteration 2271 / 88550) loss: 1.057264
(Iteration 2281 / 88550) loss: 1.114834
(Iteration 2291 / 88550) loss: 1.126265


KeyboardInterrupt: 

In [9]:

y_pred_valid = model( threeD_to_fourDTensor(X_valid).float())
val_acc = get_accuracy(y_pred_valid, Y_valid,
    batch_size=X_valid.shape[0])
print('validation accuracy:', val_acc)

RuntimeError: $ Torch: not enough memory: you tried to allocate 3GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

In [23]:
#Calculating the validation accruacy in baaches, since out-of-memory error arises when calculating using the whole set!
datset_size =  X_valid.shape[0]
size_of_batches = int(datset_size/500)

print(datset_size)
print(size_of_batches)

start = 0

valid_accuracies = []
for i in range(500):
    end = start + size_of_batches
    
    y_pred_valid = model( threeD_to_fourDTensor(X_valid[start:end]).float())
    val_acc = get_accuracy(y_pred_valid, Y_valid[start:end],
    batch_size=len(Y_valid[start:end]))
    start = end
    valid_accuracies.append(val_acc)

print('validation accuracy:', np.mean(valid_accuracies))

87250
174
validation accuracy: 37.1183908045977


In [25]:
#Calculating the training accruacy in batches, since out-of-memory error arises when calculating using the whole set!
datset_size =  X_train.shape[0]
size_of_batches = int(datset_size/500)

print(datset_size)
print(size_of_batches)

start=end = 0

train_accuracies = []
for i in range(500):
    end = start + size_of_batches
    
    y_pred_train = model( threeD_to_fourDTensor(X_train[start:end]).float())
    train_acc = get_accuracy(y_pred_train, Y_train[start:end],
    batch_size=len(Y_train[start:end]))
    start = end
    train_accuracies.append(train_acc)

print('train accuracy:', np.mean(train_accuracies))


177125
354
train accuracy: 47.509604519774015


In [26]:
#Calculating the test accruacy in baaches, since out-of-memory error arises when calculating using the whole set!
datset_size =  X_test.shape[0]
size_of_batches = int(datset_size/500)

print(datset_size)
print(size_of_batches)

start = 0

test_accuracies = []
for i in range(500):
    end = start + size_of_batches
    y_pred_test = model( threeD_to_fourDTensor(X_test[start:end]).float())
    test_acc = get_accuracy(y_pred_test, Y_test[start:end],
    batch_size=len(Y_test[start:end]))
    start = end
    test_accuracies.append(test_acc)

print('Test accuracy:', np.mean(test_accuracies))

55375
110
Test accuracy: 35.74


In [20]:
X_test_tensor = threeD_to_fourDTensor(X_test)
y_pred_test = model( X_test_tensor.float())
test_acc = get_accuracy(y_pred_test, Y_test,
    batch_size=X_test.shape[0])
print('test accuracy:', test_acc)

RuntimeError: $ Torch: not enough memory: you tried to allocate 11GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

In [ ]:
import matplotlib.pyplot as plt
plt.plot(iterations,train_loss)
plt.show();